In [1]:
    from unittest import TestCase, expectedFailure, skipUnless
    try:
        from IPython import get_ipython
        from IPython.utils.capture import capture_output
    except: 
        get_ipython = None
    from importnb import Notebook, reload, Partial, load_ipython_extension, unload_ipython_extension

In [2]:
    class TestExtension(TestCase): 
        def setUp(Test): load_ipython_extension()
                
        def test_import(Test):
            try:
                from . import loader
            except:
                import loader

            assert loader.__file__.endswith('.ipynb')
            assert loader.test is 42
            assert isinstance(loader, __import__('types').ModuleType)
            
        @expectedFailure
        def test_failure(Test):
            from . import failure
            
        def tearDown(Test): unload_ipython_extension()


In [3]:
    class TestContext(TestCase): 
        def setUp(Test): 
            with Notebook(): 
                try:
                    from . import loader
                except:
                    import loader
                Test.loader = loader
                  
        def test_import(Test):
            loader = Test.loader
            assert loader.__file__.endswith('.ipynb')
            assert loader.test is 42
            assert isinstance(loader, __import__('types').ModuleType)
            
        @expectedFailure
        def test_reload_without_context(Test):
            reload(Test.loader)
            
        def test_reload_with_context(Test):
            with Notebook(): assert reload(Test.loader)

In [4]:
    class TestPartial(TestCase): 
        def setUp(Test): 
            with Partial(capture=True): 
                try:
                    from . import failure
                except:
                    import failure
                Test.failure = failure
                  
        def test_exception(Test):
            assert Test.failure.a is 42
            assert isinstance(Test.failure.__exception__, BaseException)
            assert not hasattr(Test.failure, 'b')
            
        def test_traceback(Test):
            from traceback import print_tb
            from io import StringIO
            s = StringIO()
            print_tb(Test.failure.__exception__.__traceback__, file=s)
            assert """tests/failure.ipynb", line 22, in <module>\n    "    assert False\\n""" in s.getvalue(), """Traceback is not satisfied"""

In [5]:
    class TestRemote(TestCase): 
        @skipUnless(get_ipython and get_ipython(), """requires IP""")
        def test_imports(Test):
            with capture_output(), Notebook():
                from tests.PythonDataScienceHandbook.notebooks import _02_The_Basics_Of_NumPy_Arrays as tutorial
            assert all(hasattr(tutorial, str) for str in ('grid', 'x', 'x1', 'x2'))